### Домашнее задание "Проблема качества данных"

In [164]:
import numpy as np
import pandas as pd
import seaborn as sns
import random
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

Допустим у нас есть датасет квартир, который выставлены на продажу.

In [165]:
n_samples = 1000
# Чтобы упростить задачу, создадим признаки примерно одной размерности

n_entrance = np.random.choice(9, n_samples) + 2 #кол-во подъездов в доме
n_owners = np.random.choice(5, n_samples) + 1    #кол-во собственников квартиры
dist = np.random.choice(10, n_samples) + 1       #расстояние до центра города, км
price = np.random.choice(25, n_samples) + 5    #стоимость квадратного метра, тыс. руб.

#### Целевая переменная - срок продажи квартиры в днях

In [166]:
term = 55*n_owners/dist + 5*price + 26

In [167]:
data = pd.DataFrame({'n_entrance': n_entrance, 'n_owners': n_owners, 'dist': dist,'price':price, 'term':term})
data.head()

,n_entrance,n_owners,dist,price,term
0,8,5,8,13,125.375000
1,2,2,2,5,106.000000
2,7,3,2,11,163.500000
3,4,2,9,18,128.222222
4,4,2,3,24,182.666667


In [168]:
X = data[['n_entrance', 'n_owners', 'dist', 'price']]
y = data[['term']]
model = LinearRegression().fit(X,y)
print(f'Weigth: {model.coef_}')
print(f'Bias: {model.intercept_}')

pred = model.predict(data[['n_entrance', 'n_owners', 'dist', 'price']])
print(f'Error: {mean_absolute_error(pred,y)}')

Weigth: [[  0.14451101  16.17664525 -12.26806633   5.11156597]]
Bias: [91.72126035]
Error: 24.070855861270605


#### Попробуем убрать лишний столбец n_entrance

In [169]:
X = data[['n_owners', 'dist', 'price']]
y = data[['term']]
model = LinearRegression().fit(X,y)
print(f'Weigth: {model.coef_}')
print(f'Bias: {model.intercept_}')

pred = model.predict(data[['n_owners', 'dist', 'price']])
print(f'Error: {mean_absolute_error(pred,y)}')

Weigth: [[ 16.17869277 -12.26928749   5.11208654]]
Bias: [92.59069505]
Error: 24.081929454355926


#### Видим, что на качество модели столбец n_entrance не имел влияния, значит, в датасет его можно не включать.
#### Попробуем получить новый признак используя столбец dist, предположив, что расстояние увеличивает срок продажи квартиры.

In [170]:
data['dist_'] = 1/data['dist']

In [171]:
X = data[['n_owners', 'dist_', 'price']]
y = data[['term']]
model = LinearRegression().fit(X,y)
print(f'Weigth: {model.coef_}')
print(f'Bias: {model.intercept_}')

pred = model.predict(data[['n_owners', 'dist_', 'price']])
print(f'Error: {mean_absolute_error(pred,y)}')

Weigth: [[ 15.75859668 172.30698323   5.01535152]]
Bias: [-22.59306957]
Error: 12.74030657964326


#### Ошибка уменьшилась, значит предположение было верное. Попробуем заменить признаки n_owners и dist_ на их произведение и проверить как новый признак повлияет на качество модели.


In [172]:
data['owners_dist'] = data['n_owners']/data['dist']

In [173]:
X = data[['owners_dist', 'price']]
y = data[['term']]
model = LinearRegression().fit(X,y)
print(f'Weigth: {model.coef_}')
print(f'Bias: {model.intercept_}')

pred = model.predict(data[['owners_dist', 'price']])
print(f'Error: {mean_absolute_error(pred,y)}')

Weigth: [[55.  5.]]
Bias: [26.]
Error: 1.269739868803299e-14


#### Получили ошибку близкую к нулю.